# Análisis Exploratorio de Datos (EDA - Sobre los datos originales):

Se tiene a discposición 3 archivos en formato ".json" comprimidos (steam_games, user_reviews y user_items) guardados en la carpeta "Original_Data". En este análisis preliminar solo se abrirán los archivos, se revisará la estructura de los datos, que tipo de formato tiene cada columna y si hay registros vacíos. También se verificará que campos pueden usarse para relacionar a los diferentes datasets entre sí. 

### 1. Importación de librerías para abrir los documentos:

In [1]:
import gzip                             # Para descomprimir los archivos ".json"
import pandas as pd                     # Para convertir los archivos en dataframes
import json                             # Para leer los documentos ".json"
import ast                              # Para abrir los archivos con error en el formato ".json"
import chardet                          # Para encontrar el encoding de los archivos
import warnings                         # Para Ignorar errores
warnings.filterwarnings("ignore")                           

### 2. Fórmula para descomprimir y abrir los archivos:

Debido a que algunas líneas presentaban errores, no fue sencillo abrir leer los archivos así que se opta por una función que lea los archivos comprimidos ".json", ignores e imprima los errores.

In [2]:
# Función para abrir los archivos:

def abrir_comprimido(archivo):
    
    data_list = []
    with gzip.open(archivo, 'rt', encoding='utf-8') as file:
        for line in file.readlines():
            try:
                data = json.loads(line.strip())
                data_list.append(data)
            except json.JSONDecodeError as e:
                print(f"Error en línea: {line.strip()}")
                print(e)
    return data_list



### 3. Análisis Preliminar del archivo "steam_games.json.gz":

In [3]:
# Abrir archivo y convertirlo en datafame:

archivo = 'Original_Data\steam_games.json.gz'
df_Steam_Games = pd.DataFrame(abrir_comprimido(archivo)) # uso de la fórmula creada para descomprimir y leer archivo.

In [4]:
# Visualizar dataframe:

df_Steam_Games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se aprecia que las primeras líneas contienen registros vacíos. A continuación se procede a revisar cuantos registros pueden estar vacíos:

In [5]:
# Muestra la cuenta de NaN por columnas:
print(df_Steam_Games.isna().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


La idea inicial de este análisis preliminar no incluía imputación o eliminación de registros, pero debido a que representan la mayorías de los datos se procede a eliminar estos registros:

In [6]:
df_Steam_Games.dropna(how='all',inplace=True) # Eliminar registros que contienen vacíos en todas las columnas
print(df_Steam_Games.isna().sum()) # Muestra la cuenta de NaN por columnas

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


Se reduce ampliamente los registros del dataset, que eal parecer en su mayoría eran vacíos. A continuación se procede a visualizar el tipo de datos de cada columna:

In [7]:
df_Steam_Games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


Se aprecia que los tipos de datos no son acordes a la información registrada. Esto será insumo para el ETL correspondiente de este archivo (cambiar tipo de dato correspondiente por columna). A continación visualizamos los datos:

In [8]:
df_Steam_Games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


Se verifica si 'id' es un identificador único:

In [9]:
# Verificar duplicados en la columna 'id'
duplicados = df_Steam_Games['id'].duplicated()

# Contar la cantidad de duplicados en la columna 'id'
cantidad_duplicados = df_Steam_Games['id'].duplicated().sum()
print(f"Cantidad de duplicados en 'id': {cantidad_duplicados}")

# Eliminar duplicados basados en la columna 'id' en el mismo DataFrame
df_Steam_Games.drop_duplicates(subset=['id'], inplace=True)

# Mostrar el DataFrame resultante sin duplicados
df_Steam_Games.info()


Cantidad de duplicados en 'id': 2
<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   genres        28850 non-null  object
 2   app_name      32131 non-null  object
 3   title         30083 non-null  object
 4   url           32133 non-null  object
 5   release_date  30066 non-null  object
 6   tags          31970 non-null  object
 7   reviews_url   32132 non-null  object
 8   specs         31463 non-null  object
 9   price         30756 non-null  object
 10  early_access  32133 non-null  object
 11  id            32132 non-null  object
 12  developer     28834 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


In [10]:
df_Steam_Games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com


### Observaciones:
- Las columnas app_name y title parecen contener la misma información, se revisará en el ETL la posibilidad de imputar valores faltantes entre ellas y eliminar una columna.
- La columna tags contiene información de genre y Price (cuando es Free to Play). Esto se usará para imputar valores en ambas columnas y eliminar tags.
- Url y review url no paracen contener información relevante ni para el modelo de recomendación, ni para los endpoints de la API. Se eliminarán.
- No se eliminarán filas ni se imputarpan datos luego de realizado lo anterior, dado que para cada caso específico se manejarán filtros (consultas) de las columnas relevantes para cada enpoint o el modelo.

A continuación se exporta el datafame a formato CSV, para ser trabajado posteriormente en los ETL con las observaciones anteriores. El archivo se guardará en formato CSV en la carpeta Revised_Data:

In [11]:
# Exportar el DataFrame a CSV
df_Steam_Games.to_csv('Revised_Data/Steam_Games.csv', index=False)

print(f"DataFrame exportado a {'Revised_Data/Steam_Games.csv'}")

DataFrame exportado a Revised_Data/Steam_Games.csv


### 4. Análisis Preliminar del archivo "user_reviews.json.gz":

La función para abrir este archivo no funcionó bien porque los datos del archivo ".json" no continen el formato adecuado, según se aprecia en el error los datos (Clave - Valor) no están encerrados en doble comillas (") por lo que fué necesario buscar otro código que lea esta estructura. Se encuentra que "ast.literal" nos ayuda a leer cada línea como una expresión literal de python y lo agrega a una lista. Antes de usar, es necesario conocer el "encoding" del archivo para evitar errores de lectura.

In [12]:
# Función para encontrar encoding:

def detect_encoding(file_path):
        
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']
    

Se ingresa la dirección del archivo ".json" descomprimido para buscar el encoding del mismo:

In [13]:
# Ejecución de la función para detectar encoding en el archivo:

file_path = 'Original_Data/australian_user_reviews.json'
encoding_UR = detect_encoding(file_path)

print(f"La codificación del archivo es: {encoding_UR}")  

La codificación del archivo es: MacRoman


Con el encoding conocido se procede a abrir el archivo en modo lectura para agragar cada fila en "data_list" para finalmente convertirlo en un dataframe:

In [14]:
# Leyendo los datos de "australian_user_reviews.json"
data_list = []

with open ('Original_Data/australian_user_reviews.json', encoding=encoding_UR) as f:
    for line in f.readlines():
        data_list.append(ast.literal_eval(line))

df_User_Review = pd.DataFrame(data_list)
df_User_Review.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Se verifica si hay datos nulos en el archivo, encontrando que no hya registros vacíos:

In [15]:
# Muestra la cuenta de NaN por columnas:
print(df_User_Review.isna().sum())
print('----------------------------------------------|')
print(df_User_Review.info())

user_id     0
user_url    0
reviews     0
dtype: int64
----------------------------------------------|
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB
None


Se observa que la columna "reviews" contiene datos anidados:

In [16]:
df_User_Review['reviews'].iloc[0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

### Observaciones:
- La columna de url es innecesaria para la realización de los diferentes enpoints y el modelo de recomendación.
- La columna "reviews" debe expandirse conservando el "user_id".

A continuación se exporta el datafame a formato CSV, para ser trabajado posteriormente en los ETL con las observaciones anteriores. El archivo se guardará en formato CSV en la carpeta Revised_Data:

In [17]:
# Exportar el DataFrame a CSV
df_User_Review.to_csv('Revised_Data/User_Review.csv', index=False)

print(f"DataFrame exportado a {'Revised_Data/User_Review.csv'}")

DataFrame exportado a Revised_Data/User_Review.csv


### 4. Análisis Preliminar del archivo "users_items.json.gz":

Del mismo modo que el archivo anterior, los datos separados por comillas sencillas complican que el archivo se abra fácilmente, así que se hace uso de "ast.literal":

In [18]:
# Ejecución de la función para detectar encoding en el archivo:

#file_path = 'Original_Data/australian_users_items.json'
#encoding_UI = detect_encoding(file_path)

#print(f"La codificación del archivo es: {encoding_UI}")  

In [18]:
# Leyendo los datos de "australian_user_reviews.json"
data_list_UI = []

with open ('Original_Data/australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        data_list_UI.append(ast.literal_eval(line))

df_User_Items = pd.DataFrame(data_list_UI)
df_User_Items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Se verifica si hay datos nulos en el archivo, encontrando que no hay registros vacíos:

In [19]:
# Muestra la cuenta de NaN por columnas:
print(df_User_Items.isna().sum())
print('----------------------------------------------|')
print(df_User_Items.info())

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64
----------------------------------------------|
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB
None


Se observa que la columna "items" contiene datos anidados:

In [20]:
df_User_Items['items'].iloc[0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

### Observaciones:
- La columna de url es innecesaria para la realización de los diferentes enpoints y el modelo de recomendación.
- La columna "steam_id" parece representar el id del video juego, pero debe revisarse si tiene relación con id en el dataset de Steam_Games porque al parecer en dicho archivo "id" solo representa una parte del "steam_id".
- La columna "items_Count" podría eliminarse ya que de necesitar dicha consulta debería ser reproducible de contar los ítems anidados en la columna "items".
- Se debe extender la columna "items" porque se encuentra anidada.

A continuación se exporta el datafame a formato CSV, para ser trabajado posteriormente en los ETL con las observaciones anteriores. El archivo se guardará en formato CSV en la carpeta Revised_Data:

In [21]:
# Exportar el DataFrame a CSV
df_User_Items.to_csv('Revised_Data/User_Items.csv', index=False)

print(f"DataFrame exportado a {'Revised_Data/User_Items.csv'}")

DataFrame exportado a Revised_Data/User_Items.csv


No se realiza eliminación de registros (filas) más allá de las filas comppletamente vacías, dado que para cada endpoint y el modelo podría tenerse una importancia muy selectiva de las columnas, lo que significa que estas elimnaciones pueden manejarse a traves de filtros o cosnultas específicas para cada función.